In [1]:
import requests
import json
import pandas as pd
import numpy as np
import copy
import time

In [2]:
URL = 'http://localhost:8983/solr/craving/'

## Product Recommendation

In [3]:
def print_results(dict_returns, num_returns, query):
    '''
    dict_returns: dictionary converted from JSON results (after ranking)
    return_limit: number of returned results
    query: input for Solr
    '''
    print()
    print('Here are the top %s healthy options for "%s":' %(str(num_returns), query))
    for doc in dict_returns['response']['docs'][:num_returns]:
        print('******************************************')
        print(doc['fields.brand_name'][0], doc['fields.item_name'][0])
        print('   Nutrition Facts per serving (%s %s):' %(doc['fields.nf_serving_size_qty'][0],
                                                          doc['fields.nf_serving_size_unit'][0]))
        print('\tSugar:', doc['fields.nf_sugars'][0])
        print('\tSodium:', doc['fields.nf_sodium'][0])
        print('\tSaturated fat:', doc['fields.nf_saturated_fat'][0])
        print('\tDietary fiber:', doc['fields.nf_dietary_fiber'][0])
        print('\tProtein:', doc['fields.nf_protein'][0])
        print()

In [18]:
def query_with_pagination(query='*', return_limit=5000):
    '''
    Query with pagination, returns ranked results.
    '''
    offset = 0
    dict_returns = {}
    while True:
        payload = json.dumps({
        'query': query,
        'filter': 'fields.nf_sugars:[* TO *], fields.nf_sodium:[* TO *], fields.nf_saturated_fat:[* TO *], \
                   fields.nf_dietary_fiber:[* TO *], fields.nf_protein:[* TO *]',
        'offset': offset,
        'limit': return_limit,
        'sort': 'fields.nf_sugars asc, fields.nf_sodium asc, fields.nf_saturated_fat asc, \
                 fields.nf_dietary_fiber desc, fields.nf_protein desc',
        'params':{'q.op': 'AND'}
        })
        headers = {'Content-Type': 'application/json'}
        response = requests.request("POST", URL+'query', headers=headers, data=payload)
        page_returns = response.json()
        if offset == 0:
            dict_returns = copy.deepcopy(page_returns)
        else:
            dict_returns['response']['docs'].extend(page_returns['response']['docs'])
            dict_returns['response']['numFound'] += len(page_returns['response']['docs'])        
        if len(page_returns['response']['docs']) < return_limit:
            break
        else:
            offset += return_limit
    return dict_returns

DICT_RETURNS_NO_QUERY = query_with_pagination(query='*')

In [58]:
def filter_with_ids(dict_returns, id_limit=[]):
    '''
    Filter result with id_limit
    '''
    id_limit = set(id_limit)
    if len(id_limit) > 0:
        print('Filtering query results...')
        time_start = time.time()
        dict_filter = copy.deepcopy(dict_returns)
        dict_filter['response']['docs'] = []
        dict_filter['response']['numFound'] = 0
        
        for doc in dict_returns['response']['docs']:
            if doc['item_id'][0] in id_limit:
                dict_filter['response']['docs'].append(doc)
                dict_filter['response']['numFound'] += 1

        dict_returns = copy.deepcopy(dict_filter)
        
    return dict_returns

In [59]:
def product_recommend(query='*', num_options=3, return_limit=5000, id_limit=[], preference=[], allergy=[]):
    '''
    Given a product, find healthy options
    Criteria: low sugar, low sodium, low unsaturated fats
              high fiber, high protein
    Input: query - string, no field name required
           num_options: number of recommendations desired
           return_limit: number of products listed per page
           id_limit: list of item ids for filtering ranked results
           preference: list of user dietary preferences
           allergy: list of user allergies
    '''
    # Query in Solr 
    print('Gathering query results...')
    time_start = time.time()
    if query == '*':
        dict_returns = DICT_RETURNS_NO_QUERY
    else:
        dict_returns = query_with_pagination(query=query, return_limit=return_limit)
    print('Query time:', time.time() - time_start)
    
    # filter with id_limit   
    dict_returns = filter_with_ids(dict_returns, id_limit=id_limit)
    print('Filtering time:', time.time() - time_start)
    
    # print results
    num_found = dict_returns['response']['numFound']
    if num_found > 0:
        print_results(dict_returns, min(num_options, num_found), query)
    else:
        print('The query "%s" did not match any products.' % query)

### Tests

In [60]:
product_recommend(query='kind banana')

Gathering query results...
Query time: 0.010746002197265625
Filtering time: 0.010746002197265625

Here are the top 3 healthy options for "kind banana":
******************************************
Kind Banana Nut Clusters
   Nutrition Facts per serving (0.33 cup):
	Sugar: 3
	Sodium: 35
	Saturated fat: 0
	Dietary fiber: 2
	Protein: 2.0

******************************************
Kind Dark Chocolate Peanut Butter Banana Snack Mix
   Nutrition Facts per serving (1.0 pack):
	Sugar: 3
	Sodium: 35
	Saturated fat: 2
	Dietary fiber: 1
	Protein: 3.0

******************************************
Kind Dark Chocolate Peanut Butter Banana Snack Mix
   Nutrition Facts per serving (0.25 cup):
	Sugar: 5
	Sodium: 55
	Saturated fat: 3
	Dietary fiber: 2
	Protein: 5.0



In [61]:
# this takes about 20 seconds
product_recommend(id_limit=["5b28aae078a76828637cecc7", "5a87d4ac7c76309a03c87cb1"])

Gathering query results...
Query time: 0.0
Filtering query results...
Filtering time: 6.2321813106536865

Here are the top 2 healthy options for "*":
******************************************
Kind Breakfast Protein Bar, Peanut Butter Banana Dark Chocolate
   Nutrition Facts per serving (2.0 bars):
	Sugar: 10
	Sodium: 135
	Saturated fat: 2
	Dietary fiber: 4
	Protein: 8.0

******************************************
Kind Pressed, Fruit Bars, Pineapple Banana Kale Spinach
   Nutrition Facts per serving (1.0 bar):
	Sugar: 18
	Sodium: 45
	Saturated fat: 0
	Dietary fiber: 2
	Protein: 1.0

